In [1]:
from cophi_toolbox import preprocessing
from dariah_topics import postprocessing
from dariah_topics import visualization
import metadata_toolbox.utils as metadata
from pathlib import Path
import pandas as pd
import lda
import os

In [2]:
#path_to_corpus = Path('data', 'grenzboten_sample')
path_to_corpus = Path('/media/sf_VBox_Shared/Arabic/Fiqh/2018-11-06-chapters-khoja-txt-renamed/')
#pattern = '{author}_{year}_{title}'
pattern = '{year}.{author}.{title}.{chapter}'

In [3]:
meta = pd.concat([metadata.fname2metadata(str(path), pattern=pattern) for path in path_to_corpus.glob('*.txt')])
#meta = pd.concat(meta_list)
meta[:5] # by adding '[:5]' to the variable, only the first 5 elements will be printed

,year,author,title,chapter
/media/sf_VBox_Shared/Arabic/Fiqh/2018-11-06-chapters-khoja-txt-renamed/1319.IbnYasinLabadiNabulusi.HashiyatCalaNaylMaarib.00123.txt,1319,IbnYasinLabadiNabulusi,HashiyatCalaNaylMaarib,00123
/media/sf_VBox_Shared/Arabic/Fiqh/2018-11-06-chapters-khoja-txt-renamed/1319.IbnYasinLabadiNabulusi.HashiyatCalaNaylMaarib.00124.txt,1319,IbnYasinLabadiNabulusi,HashiyatCalaNaylMaarib,00124
/media/sf_VBox_Shared/Arabic/Fiqh/2018-11-06-chapters-khoja-txt-renamed/1319.IbnYasinLabadiNabulusi.HashiyatCalaNaylMaarib.00125.txt,1319,IbnYasinLabadiNabulusi,HashiyatCalaNaylMaarib,00125
/media/sf_VBox_Shared/Arabic/Fiqh/2018-11-06-chapters-khoja-txt-renamed/1319.IbnYasinLabadiNabulusi.HashiyatCalaNaylMaarib.00126.txt,1319,IbnYasinLabadiNabulusi,HashiyatCalaNaylMaarib,00126
/media/sf_VBox_Shared/Arabic/Fiqh/2018-11-06-chapters-khoja-txt-renamed/1319.IbnYasinLabadiNabulusi.HashiyatCalaNaylMaarib.00127.txt,1319,IbnYasinLabadiNabulusi,HashiyatCalaNaylMaarib,00127


In [4]:
len(meta)

5189

In [5]:
corpus = list(preprocessing.read_files(meta.index))
corpus[0][:255] # printing the first 255 characters of the first document

'بوب قتل بغي قول له علا قتل حجر جمع عبر عني وثن في قول صلح بين عبر لفظ نزل في قضي هي أن نبأ صلي له عليه سلم ركب حمر مرر على أبن أبي بول حمر فسد أبن أبي أنف قول أبن روح له بول حمر طوب روح من سكك فكان بين قوم ضرب يود نعل سعف اه جلل إن قتت طوف من أمن عصب أو ر'

In [6]:
tokenized_corpus = [list(preprocessing.tokenize(document)) for document in corpus]
tokenized_corpus[0][0:13]

['بوب',
 'قتل',
 'بغي',
 'قول',
 'له',
 'علا',
 'قتل',
 'حجر',
 'جمع',
 'عبر',
 'عني',
 'وثن',
 'في']

In [7]:
len(tokenized_corpus)

5189

In [8]:
document_term_matrix, document_ids, type_ids = preprocessing.create_document_term_matrix(tokenized_corpus, meta['title']+meta['chapter'], large_corpus=True)

/home/dafne/anaconda3/envs/adh/lib/python3.7/site-packages/cophi_toolbox-0.1.1.dev0-py3.7.egg/cophi_toolbox/preprocessing.py:625: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [9]:
document_term_matrix.shape, len(document_ids), len(type_ids)

((2841914, 1), 5189, 66010)

In [26]:
stopwords = preprocessing.list_mfw(document_term_matrix, most_frequent_tokens=100, type_ids=type_ids)

In [27]:
hapax_legomena = preprocessing.find_hapax_legomena(document_term_matrix, type_ids=type_ids)
print("Total number of types in corpus:", document_term_matrix.shape[1])
print("Total number of hapax legomena:", len(hapax_legomena))

Total number of types in corpus: 1
Total number of hapax legomena: 36728


In [28]:
#path_to_stopwordlist = Path('data', 'stopwords', 'de.txt')
path_to_stopwordlist = Path('/media/sf_VBox_Shared/Arabic/arabic_stop-words_7-8-2018.txt')
external_stopwords = [line.strip() for line in path_to_stopwordlist.open('r', encoding='utf-8')]

In [29]:
features = stopwords + hapax_legomena + external_stopwords
document_term_matrix = preprocessing.remove_features(features, document_term_matrix=document_term_matrix, type_ids=type_ids)

In [30]:
type_to_words = {type_ids[k]: k for k in type_ids}
docid_to_doc = {document_ids[k]: k for k in document_ids}

document_term_matrix_2 = document_term_matrix.reset_index()
document_term_matrix_2['document_id'] = [docid_to_doc[d] for d in document_term_matrix_2['document_id']]
document_term_matrix_2['type_id'] = [type_to_words[t] for t in document_term_matrix_2['type_id']]
document_term_matrix_2 = document_term_matrix_2.set_index(['document_id', 'type_id'])

document_term_matrix_2.to_csv('/media/sf_VBox_Shared/Arabic/Analyses/topicmodelling/doc_term_matrix.csv')

In [31]:
print(len(set(document_term_matrix.index.get_level_values('document_id'))))
print(len(set(document_term_matrix.index.get_level_values('type_id'))))

5189
29067


In [100]:
print(len(type_ids), len(type_to_words))

66010 66010


In [33]:
import scipy.sparse
import numpy as np

In [35]:
doc_ids_total = np.array(document_term_matrix_2.index.get_level_values('document_id'))
word_ids_total = np.array(document_term_matrix_2.index.get_level_values('type_id'))

In [36]:
doc_ids = np.sort(list(set(doc_ids_total)))
word_ids = np.sort(list(set(word_ids_total)))

doc_ids_dict = {d: i for i, d in enumerate(doc_ids)}
word_ids_dict = {d: i for i, d in enumerate(word_ids)}

In [38]:
icord = np.array([doc_ids_dict[d] for d in doc_ids_total])
jcord = np.array([word_ids_dict[d] for d in word_ids_total])
values = document_term_matrix_2.values.flatten()
print(icord.shape, jcord.shape, values.shape)

(2157597,) (2157597,) (2157597,)


In [39]:
max(icord), max(jcord)

(5188, 29066)

In [42]:
#vocabulary = [type_to_words[d] for d in doc_ids]
vocabulary = list(word_ids)
print(len(vocabulary))
vocabulary[:10]

29067


['ءاتنا', 'ءامنوا', 'ءذا', 'ءن', 'أبب', 'أبد', 'أبر', 'أبرق', 'أبض', 'أبط']

In [43]:
document_term_matrix_arr = scipy.sparse.coo_matrix((values, (icord, jcord)))
document_term_matrix_arr.shape

(5189, 29067)

In [ ]:
%%time
n_topics = 10
n_iter = 1000
model = lda.LDA(n_topics=n_topics, n_iter=n_iter)
model.fit(document_term_matrix_arr)

/home/dafne/anaconda3/envs/adh/lib/python3.7/site-packages/lda-1.1.0-py3.7-linux-x86_64.egg/lda/utils.py:55: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if sparse and not np.issubdtype(doc_word.dtype, int):


In [ ]:
fpath_out = '/media/sf_VBox_Shared/Arabic/Analyses/topicmodelling/'

In [ ]:
model.doc_topic_.tofile(os.path.join(fpath_out, 'lda{}_{}.npy'.format(n_topics, n_iter)))

In [ ]:
topics = postprocessing.show_topics(model=model,
                                    vocabulary=vocabulary)
topics

In [ ]:
topics.to_csv(os.path.join(fpath_out, 'topics.csv'))

In [ ]:
document_topics = postprocessing.show_document_topics(model=model,
                                                      topics=topics,
                                                      document_labels=meta['title'])
document_topics.to_csv(fpath_out, 'document_topics.csv')

# Visualizations

### `PlotDocumentTopics`

In [130]:
from bokeh.io import output_notebook, show
output_notebook()
%matplotlib inline

Loading BokehJS ...

In [132]:
PlotDocumentTopics = visualization.PlotDocumentTopics(document_topics)

### `static_heatmap`

In [ ]:
static_heatmap = PlotDocumentTopics.static_heatmap(colorbar=False)

### `static_barchart_per_topic`

In [ ]:
static_barchart_per_topic = PlotDocumentTopics.static_barchart_per_topic(index=0, # or index='abgewiesen südlich genommen'
                                                                         describer='Topic',
                                                                         alpha=None,
                                                                         figsize=(11, 7))

### `static_barchart_per_document`

In [ ]:
static_barchart_per_document = PlotDocumentTopics.static_barchart_per_document(index=0,
                                                                               describer='Document')

### `interactive_heatmap`

In [ ]:
interactive_heatmap = PlotDocumentTopics.interactive_heatmap(width=800,
                                                             height=550,
                                                             colorbar=False)
show(interactive_heatmap, notebook_handle=True)

### `interactive_barchart_per_topic`

In [ ]:
interactive_barchart_per_topic = PlotDocumentTopics.interactive_barchart_per_topic(index=0,
                                                                                   describer='Topic',
                                                                                   width=800)
show(interactive_barchart_per_topic, notebook_handle=True)

### `interactive_barchart_per_document`

In [ ]:
interactive_barchart_per_document = PlotDocumentTopics.interactive_barchart_per_document(index=0,
                                                                                         describer='Document',
                                                                                         width=800)
show(interactive_barchart_per_document, notebook_handle=True)